In [124]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""
#%%

import os
import numpy as np
import pandas as pd 
import random as rn
import matplotlib.pyplot as plt; 
 
# Importing sklearn libraries
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
 
# Importing hypopt library for grid search
#from hypopt import GridSearch
 
# Importing Keras libraries
from keras.utils import np_utils
from keras.models import Sequential
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D
from keras.models import Model
import urllib
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from keras import backend as K

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


#for multiple outs from single cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

'\nSpyder Editor\n\nThis is a temporary script file.\n'

In [137]:
img_dir = './MobileImages/data4ImageGen/'
#Rename Files in Each Folder if necessary & sync to single folder trainData
#find minor/ -type f -exec  sh -c 'mv {} $(dirname {})/$(basename $(dirname {}))_$(basename {})' \;
#rsync -a moderate/ trainData/
train_datagen = ImageDataGenerator(
    rotation_range=2,
    #width_shift_range=0.25,
    #height_shift_range=0.25,
    rescale=1/255,
    #shear_range=0.22,
    zoom_range=0.25,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.25
)
test_datagen = ImageDataGenerator(
    rescale=1/255,
)

# I found that a batch size of 128 offers the best trade-off between
# model training time and batch volatility.
batch_size = 50

# Notice the tiny target size, just 48x48!
train_generator = train_datagen.flow_from_directory(
    img_dir + 'training/',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical'
)
validation_generator = train_datagen.flow_from_directory(
    img_dir + 'validation/',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical'
)

# define the model
prior = keras.applications.VGG16(
    include_top=False, 
    weights='imagenet',
    input_shape=(48, 48 , 3)
)
model = Sequential()
model.add(prior)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.04))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.03))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.02))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.01))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax', name='Output'))
 
# freeze the vgg16 model
for cnn_block_layer in model.layers[0].layers:
    cnn_block_layer.trainable = False
model.layers[0].trainable = False


# compile the model
model.compile(
    optimizer="adam",
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# fit the model
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator.filenames) // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=len(train_generator.filenames) // batch_size,
    callbacks=[
        EarlyStopping(patience=7 , restore_best_weights=True)
    ]
)


# save model artifact
model.save('./SavedModels/model-48.h5')


Found 418 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
Epoch 1/20
8/8 [==============================] - 17s 2s/step - loss: 0.6779 - acc: 0.6024 - val_loss: 0.7036 - val_acc: 0.5550
Epoch 2/20
8/8 [==============================] - 8s 1s/step - loss: 0.6657 - acc: 0.5973 - val_loss: 0.6647 - val_acc: 0.5575
Epoch 3/20
8/8 [==============================] - 10s 1s/step - loss: 0.6470 - acc: 0.6283 - val_loss: 0.6045 - val_acc: 0.7025
Epoch 4/20
8/8 [==============================] - 8s 956ms/step - loss: 0.5578 - acc: 0.7228 - val_loss: 0.5899 - val_acc: 0.7200
Epoch 5/20
8/8 [==============================] - 10s 1s/step - loss: 0.5224 - acc: 0.7368 - val_loss: 0.5618 - val_acc: 0.7300
Epoch 6/20
8/8 [==============================] - 9s 1s/step - loss: 0.5228 - acc: 0.7239 - val_loss: 0.5759 - val_acc: 0.7225
Epoch 7/20
8/8 [==============================] - 9s 1s/step - loss: 0.4885 - acc: 0.7675 - val_loss: 0.6137 - val_acc: 0.7025
Epoch 8/20
8/8 [========

In [138]:
from keras.models import load_model


# define data generators
train_datagen = ImageDataGenerator(
    rotation_range=20,
    #width_shift_range=0.25,
    #height_shift_range=0.25,
    rescale=1/255,
    shear_range=0.2,
    #zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.25
)

test_datagen = ImageDataGenerator(
    rescale=1/255,
)
train_generator = train_datagen.flow_from_directory(
    img_dir + 'training/',
    target_size=(96, 96),
    batch_size=batch_size,
    class_mode='categorical'
)
validation_generator = train_datagen.flow_from_directory(
    img_dir + 'validation/',
    target_size=(96, 96),
    batch_size=batch_size,
    class_mode='categorical'
)


model = Sequential()
model.add(Conv2D(64, dilation_rate=2, kernel_size=(3, 3), input_shape=(96, 96, 3), activation='relu', padding='same'))
model.add(Dropout(0.03))
model.add(Conv2D(64, dilation_rate=2, kernel_size=(3, 3), input_shape=(96, 96, 3), activation='relu', padding='same'))
model.add(Dropout(0.03))
model.add(MaxPooling2D(pool_size=(3, 3)))

# load the pretrained model
prior = load_model('./SavedModels/model-48.h5')

# add all but the first two layers of VGG16 to the new model
# strip the input layer out, this is now 96x96
# also strip out the first convolutional layer, this took the 48x48 input and convolved it but
# this is now the job of the three new layers.
for layer in prior.layers[0].layers[2:]:
    model.add(layer)

# re-add the feedforward layers on top
for layer in prior.layers[1:]:
    model.add(layer)

# the pretrained CNN layers are already marked non-trainable mark off the top layers as well
for layer in prior.layers[-4:]:
     layer.trainable = False
    
# compile the model
model.compile(
    optimizer="adam",
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# fit the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator.filenames) // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=len(train_generator.filenames) // batch_size,
    callbacks=[
        EarlyStopping(patience=10, restore_best_weights=True)
    ]
)

# save model artifact
model.save('./SavedModels/model-96.h5')
# with open('./SavedModels/model-96-history.pickle', 'wb') as fp:
#         pickle.dump(history.history, fp)

Found 418 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
Epoch 1/20
8/8 [==============================] - 33s 4s/step - loss: 0.7776 - acc: 0.4801 - val_loss: 0.7010 - val_acc: 0.5400
Epoch 2/20
8/8 [==============================] - 27s 3s/step - loss: 0.7133 - acc: 0.5250 - val_loss: 0.6865 - val_acc: 0.5400
Epoch 3/20
8/8 [==============================] - 23s 3s/step - loss: 0.7010 - acc: 0.5207 - val_loss: 0.6761 - val_acc: 0.6725
Epoch 4/20
8/8 [==============================] - 25s 3s/step - loss: 0.6888 - acc: 0.5505 - val_loss: 0.6636 - val_acc: 0.6825
Epoch 5/20
8/8 [==============================] - 26s 3s/step - loss: 0.6869 - acc: 0.5058 - val_loss: 0.6532 - val_acc: 0.6250
Epoch 6/20
8/8 [==============================] - 27s 3s/step - loss: 0.6641 - acc: 0.5825 - val_loss: 0.6370 - val_acc: 0.6275
Epoch 7/20
8/8 [==============================] - 25s 3s/step - loss: 0.6517 - acc: 0.6031 - val_loss: 0.6282 - val_acc: 0.6550
Epoch 8/20
8/8 [=======

In [143]:
from keras.models import load_model


# define data generators
train_datagen = ImageDataGenerator(
    rotation_range=20,
    #width_shift_range=0.25,
    #height_shift_range=0.25,
    rescale=1/255,
    shear_range=0.2,
    #zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.25
)

test_datagen = ImageDataGenerator(
    rescale=1/255,
)
train_generator = train_datagen.flow_from_directory(
    img_dir + 'training/',
    target_size=(192, 192),
    batch_size=batch_size,
    class_mode='categorical'
)
validation_generator = train_datagen.flow_from_directory(
    img_dir + 'validation/',
    target_size=(192, 192),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    directory= img_dir + 'testing/',
    target_size=(192, 192),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(192, 192, 3), activation='relu', padding='same'))
model.add(Dropout(0.02))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(Dropout(0.02))
model.add(MaxPooling2D(pool_size=(2, 2)))

# load the pretrained model
prior = load_model('./SavedModels/model-96.h5')

# add all but the first two layers of VGG16 to the new model
# strip the input layer out, this is now 96x96
# also strip out the first convolutional layer, this took the 96x96 input and convolved it but
# this is now the job of the three new layers.dany
for layer in prior.layers[1:]:
    layer.name += '_prior'  # set layer names to avoid name collisions
    model.add(layer)

# the pretrained CNN layers are already marked non-trainable
# mark off the top layers as well
for layer in prior.layers[-4:]:
    layer.trainable = False
    
# compile the model
model.compile(
    optimizer="adam",
    loss='categorical_crossentropy',
    metrics=['accuracy']
)  

# fit the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator.filenames) // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=len(train_generator.filenames) // batch_size,
    callbacks=[
        EarlyStopping(patience=10, restore_best_weights=True)
    ]
)

# save model artifact
model.save('./SavedModels/model-192.h5')
# with open('./SavedModels/model-96-history.pickle', 'wb') as fp:
#         pickle.dump(history.history, fp)


Found 418 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
Found 52 images belonging to 1 classes.
Epoch 1/20
8/8 [==============================] - 67s 8s/step - loss: 0.7133 - acc: 0.4506 - val_loss: 0.6893 - val_acc: 0.5400
Epoch 2/20
8/8 [==============================] - 56s 7s/step - loss: 0.6910 - acc: 0.5418 - val_loss: 0.6891 - val_acc: 0.5400
Epoch 3/20
8/8 [==============================] - 58s 7s/step - loss: 0.6944 - acc: 0.5126 - val_loss: 0.6914 - val_acc: 0.5375
Epoch 4/20
8/8 [==============================] - 61s 8s/step - loss: 0.6892 - acc: 0.5725 - val_loss: 0.6866 - val_acc: 0.5525
Epoch 5/20
8/8 [==============================] - 54s 7s/step - loss: 0.6889 - acc: 0.5102 - val_loss: 0.6782 - val_acc: 0.5550
Epoch 6/20
8/8 [==============================] - 62s 8s/step - loss: 0.6753 - acc: 0.5675 - val_loss: 0.6489 - val_acc: 0.6550
Epoch 7/20
8/8 [==============================] - 54s 7s/step - loss: 0.6714 - acc: 0.5717 - val_loss: 0.6246 -

In [144]:
STEP_SIZE_VALID = len(train_generator.filenames) // batch_size
model.evaluate_generator(generator=validation_generator,steps=STEP_SIZE_VALID)

[0.5740638822317123, 0.6825000122189522]

In [145]:
test_generator = test_datagen.flow_from_directory(
    img_dir + 'testing/',
    target_size=(192, 192),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)  

STEP_SIZE_TEST=len(test_generator.filenames) // test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

Found 52 images belonging to 1 classes.
52/52 [==============================] - 6s 111ms/step


In [146]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
#results.to_csv("results.csv",index=False 
results

,Filename,Predictions
0,test/Damage_327-Copy1.jpg,Damage
1,test/Damage_332-Copy1.jpg,NoDamage
2,test/Damage_335-Copy1.jpg,NoDamage
3,test/Damage_341-Copy1.jpg,NoDamage
4,test/Damage_342-Copy1.jpg,NoDamage
5,test/Damage_345-Copy1.jpg,Damage
6,test/Damage_348-Copy1.jpg,Damage
7,test/Damage_354-Copy1.jpg,NoDamage
8,test/Damage_355-Copy1.jpg,NoDamage
9,test/Damage_364-Copy1.jpg,Damage


In [74]:
preds = np.argmax(model.predict(test_features), axis=1)
print("\nAccuracy on Test Data: ", accuracy_score(test_y, preds))
print("\nNumber of correctly identified imgaes: ",
      accuracy_score(test_y, preds, normalize=False),"\n")

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 


heatmap_labels = ['NoDamage', 'Damage']

sns.heatmap(confusion_matrix(test_y, preds, labels=range(0,2)), annot=True, annot_kws={"size": 10}, 
            fmt='g', cmap='OrRd', xticklabels=heatmap_labels, yticklabels=heatmap_labels)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')

plot_acc_loss(history)

NameError: name 'test_features' is not defined

In [ ]:
def mobile_damage_estimator(image_path, model_transfer,vggModel):
    
        
    print ("Determining if damaged...")
    #urllib.urlretrieve(image_path, 'save.jpg') # or other way to upload image
    img = load_img(image_path, target_size=(224, 224)) # this is a PIL image 
    x = img_to_array(img) # this is a Numpy array with shape (3, 256, 256)
    #x = x.reshape((1,) + x.shape)/255 # this is a Numpy array with shape (1, 3, 256, 256)
    x = np.expand_dims(x, axis=0)
    
    x = imagenet_utils.preprocess_input(x)
    
    features = vggModel.predict(x, batch_size=32)
    
    pred = model_transfer.predict(features)
    pred_label = np.argmax(pred, axis=1)
    
    d = {0: 'NoDamage', 1: 'Damage'}
    for key in d.keys():
        if pred_label[0] == key:
            print ("Assessment: {} to Mobile".format(d[key]))
    print ("Assessment complete.")
    view_images(image_path)
    
def view_images(img):
    from IPython.display import Image, display, clear_output
    #clear_output()
    display(Image(img))

In [ ]:
mobile_damage_estimator('./MobileImages/test1/Damage_999.jpg', model_transfer,model)
mobile_damage_estimator('./MobileImages/test1/NoDamage_998.jpg', model_transfer,model)
